## **Import libraries and data**

In [1]:
import sklearn 
import pandas as pd  
import matplotlib.pyplot as plt
import requests
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

## Cut the request in multiple requests (month per month)

In [2]:
def generate_monthly_ranges(start_date, end_date):
    # Parse the input start and end dates
    start = datetime.strptime(start_date, "%Y-%m-%dT%H:%M")
    end = datetime.strptime(end_date, "%Y-%m-%dT%H:%M")
    
    # Initialize the first month start and end dates
    current_start = start
    
    while current_start <= end:
        # Get the last day of the current month
        current_end = (current_start + relativedelta(months=1) - timedelta(days=1)).replace(hour=23)
        
        # Ensure we don't go past the actual end date
        if current_end > end:
            current_end = end
        
        # Format as required for the API request
        yield current_start.strftime("%Y-%m-%dT%H:%M"), current_end.strftime("%Y-%m-%dT%H:%M")
        
        # Move to the next month
        current_start = (current_start + relativedelta(months=1)).replace(hour=0)


## API request

**Note AB: changed dates until october**

In [3]:
API_TOKEN = 'fb3872756c312e32fa40585dbbbbc3c156261c2bfc734f2adfc6d6cfd9c7dd93' #Requested by email
URL_BASE = 'https://api.esios.ree.es/'
ENDPOINT = 'indicators/'
start_date = '2014-01-01T00:00'
end_date = '2024-10-31T23:00'

headers = {'Host': 'api.esios.ree.es',
           'x-api-key': API_TOKEN}

### Datas other than PBF 
Price DA (Mercado SPOT): 600 \
Generación prevista Solar: 10034 \
Previsión de la producción eólica peninsular: 541 \
Generación prevista Solar térmica: 543 \
Demanda prevista: 544 \
Demanda programada: 545

The scale is hour per hour

In [4]:
### Start Parameters/

# Indicator search (look at the csv indicator_description)
indicators = ['600','10034','541','543','544','545']
indicators_name = ['price_DA', 'generacion_prevista_solar', 'prevision_produccion_eolica', 'generacion_prevista_solar_termic', 'demand_prevista', 'demand_prog']

### /End Parameters

dataframes_other_pbf = {}

for i in range(len(indicators)):
    url = URL_BASE + ENDPOINT + indicators[i]
    print(indicators[i])

    dataframes_other_pbf[indicators_name[i]]=pd.DataFrame()

    for date1, date2 in generate_monthly_ranges(start_date, end_date):

        print(date1, date2)
    
        params = {'start_date': date1,
                'end_date': date2,
                'time_agg': 'avg', #agregate through average (it's in MW)
                'time_trunc': 'hour' #aggregate to hourly level
        }

        request = requests.get(url, headers=headers, params=params)
        data = request.json()
        aux_df = pd.DataFrame(data['indicator']['values'])
        if not aux_df.empty:
            aux_df = aux_df[['datetime','datetime_utc', 'geo_id', 'geo_name', 'value']]
        dataframes_other_pbf[indicators_name[i]] = pd.concat([dataframes_other_pbf[indicators_name[i]],aux_df],ignore_index=True)

#print(dataframes_other_pbf)

600
2014-01-01T00:00 2014-01-31T23:00
2014-02-01T00:00 2014-02-28T23:00
2014-03-01T00:00 2014-03-31T23:00
2014-04-01T00:00 2014-04-30T23:00
2014-05-01T00:00 2014-05-31T23:00
2014-06-01T00:00 2014-06-30T23:00
2014-07-01T00:00 2014-07-31T23:00
2014-08-01T00:00 2014-08-31T23:00
2014-09-01T00:00 2014-09-30T23:00
2014-10-01T00:00 2014-10-31T23:00
2014-11-01T00:00 2014-11-30T23:00
2014-12-01T00:00 2014-12-31T23:00
2015-01-01T00:00 2015-01-31T23:00
2015-02-01T00:00 2015-02-28T23:00
2015-03-01T00:00 2015-03-31T23:00
2015-04-01T00:00 2015-04-30T23:00
2015-05-01T00:00 2015-05-31T23:00
2015-06-01T00:00 2015-06-30T23:00
2015-07-01T00:00 2015-07-31T23:00
2015-08-01T00:00 2015-08-31T23:00
2015-09-01T00:00 2015-09-30T23:00
2015-10-01T00:00 2015-10-31T23:00
2015-11-01T00:00 2015-11-30T23:00
2015-12-01T00:00 2015-12-31T23:00
2016-01-01T00:00 2016-01-31T23:00
2016-02-01T00:00 2016-02-29T23:00
2016-03-01T00:00 2016-03-31T23:00
2016-04-01T00:00 2016-04-30T23:00
2016-05-01T00:00 2016-05-31T23:00
2016-06-01

2023-04-01T00:00 2023-04-30T23:00
2023-05-01T00:00 2023-05-31T23:00
2023-06-01T00:00 2023-06-30T23:00
2023-07-01T00:00 2023-07-31T23:00
2023-08-01T00:00 2023-08-31T23:00
2023-09-01T00:00 2023-09-30T23:00
2023-10-01T00:00 2023-10-31T23:00
2023-11-01T00:00 2023-11-30T23:00
2023-12-01T00:00 2023-12-31T23:00
2024-01-01T00:00 2024-01-31T23:00
2024-02-01T00:00 2024-02-29T23:00
2024-03-01T00:00 2024-03-31T23:00
2024-04-01T00:00 2024-04-30T23:00
2024-05-01T00:00 2024-05-31T23:00
2024-06-01T00:00 2024-06-30T23:00
2024-07-01T00:00 2024-07-31T23:00
2024-08-01T00:00 2024-08-31T23:00
2024-09-01T00:00 2024-09-30T23:00
2024-10-01T00:00 2024-10-31T23:00
541
2014-01-01T00:00 2014-01-31T23:00
2014-02-01T00:00 2014-02-28T23:00
2014-03-01T00:00 2014-03-31T23:00
2014-04-01T00:00 2014-04-30T23:00
2014-05-01T00:00 2014-05-31T23:00
2014-06-01T00:00 2014-06-30T23:00
2014-07-01T00:00 2014-07-31T23:00
2014-08-01T00:00 2014-08-31T23:00
2014-09-01T00:00 2014-09-30T23:00
2014-10-01T00:00 2014-10-31T23:00
2014-11-01

2021-09-01T00:00 2021-09-30T23:00
2021-10-01T00:00 2021-10-31T23:00
2021-11-01T00:00 2021-11-30T23:00
2021-12-01T00:00 2021-12-31T23:00
2022-01-01T00:00 2022-01-31T23:00
2022-02-01T00:00 2022-02-28T23:00
2022-03-01T00:00 2022-03-31T23:00
2022-04-01T00:00 2022-04-30T23:00
2022-05-01T00:00 2022-05-31T23:00
2022-06-01T00:00 2022-06-30T23:00
2022-07-01T00:00 2022-07-31T23:00
2022-08-01T00:00 2022-08-31T23:00
2022-09-01T00:00 2022-09-30T23:00
2022-10-01T00:00 2022-10-31T23:00
2022-11-01T00:00 2022-11-30T23:00
2022-12-01T00:00 2022-12-31T23:00
2023-01-01T00:00 2023-01-31T23:00
2023-02-01T00:00 2023-02-28T23:00
2023-03-01T00:00 2023-03-31T23:00
2023-04-01T00:00 2023-04-30T23:00
2023-05-01T00:00 2023-05-31T23:00
2023-06-01T00:00 2023-06-30T23:00
2023-07-01T00:00 2023-07-31T23:00
2023-08-01T00:00 2023-08-31T23:00
2023-09-01T00:00 2023-09-30T23:00
2023-10-01T00:00 2023-10-31T23:00
2023-11-01T00:00 2023-11-30T23:00
2023-12-01T00:00 2023-12-31T23:00
2024-01-01T00:00 2024-01-31T23:00
2024-02-01T00:

2020-02-01T00:00 2020-02-29T23:00
2020-03-01T00:00 2020-03-31T23:00
2020-04-01T00:00 2020-04-30T23:00
2020-05-01T00:00 2020-05-31T23:00
2020-06-01T00:00 2020-06-30T23:00
2020-07-01T00:00 2020-07-31T23:00
2020-08-01T00:00 2020-08-31T23:00
2020-09-01T00:00 2020-09-30T23:00
2020-10-01T00:00 2020-10-31T23:00
2020-11-01T00:00 2020-11-30T23:00
2020-12-01T00:00 2020-12-31T23:00
2021-01-01T00:00 2021-01-31T23:00
2021-02-01T00:00 2021-02-28T23:00
2021-03-01T00:00 2021-03-31T23:00
2021-04-01T00:00 2021-04-30T23:00
2021-05-01T00:00 2021-05-31T23:00
2021-06-01T00:00 2021-06-30T23:00
2021-07-01T00:00 2021-07-31T23:00
2021-08-01T00:00 2021-08-31T23:00
2021-09-01T00:00 2021-09-30T23:00
2021-10-01T00:00 2021-10-31T23:00
2021-11-01T00:00 2021-11-30T23:00
2021-12-01T00:00 2021-12-31T23:00
2022-01-01T00:00 2022-01-31T23:00
2022-02-01T00:00 2022-02-28T23:00
2022-03-01T00:00 2022-03-31T23:00
2022-04-01T00:00 2022-04-30T23:00
2022-05-01T00:00 2022-05-31T23:00
2022-06-01T00:00 2022-06-30T23:00
2022-07-01T00:

**Note AB: Adding here to only retain spain prices, otherwise we're retrieving prices from Portugal and France**

In [5]:
prices_df = dataframes_other_pbf['price_DA']
prices_df = prices_df[prices_df['geo_name']=='España']
prices_df = prices_df.reset_index(drop=True)
dataframes_other_pbf['price_DA'] = prices_df

## Group and Select all the dataframes

Now every dataframes will be accessible thanks dataframes_dictionnary. You can access to every dataframe by his name. You can also choose to unselect some of them.

In [7]:
dataframes_dictionnary = {}
dataframes_dictionnary.update(dataframes_other_pbf)

print(dataframes_dictionnary)

{'price_DA':                             datetime          datetime_utc  geo_id geo_name  \
0      2014-01-01T00:00:00.000+01:00  2013-12-31T23:00:00Z       3   España   
1      2014-01-01T01:00:00.000+01:00  2014-01-01T00:00:00Z       3   España   
2      2014-01-01T02:00:00.000+01:00  2014-01-01T01:00:00Z       3   España   
3      2014-01-01T03:00:00.000+01:00  2014-01-01T02:00:00Z       3   España   
4      2014-01-01T04:00:00.000+01:00  2014-01-01T03:00:00Z       3   España   
...                              ...                   ...     ...      ...   
94963  2024-10-31T19:00:00.000+01:00  2024-10-31T18:00:00Z       3   España   
94964  2024-10-31T20:00:00.000+01:00  2024-10-31T19:00:00Z       3   España   
94965  2024-10-31T21:00:00.000+01:00  2024-10-31T20:00:00Z       3   España   
94966  2024-10-31T22:00:00.000+01:00  2024-10-31T21:00:00Z       3   España   
94967  2024-10-31T23:00:00.000+01:00  2024-10-31T22:00:00Z       3   España   

        value  
0       20.02  
1     

## Create CSV for all this dataframes

This can be usefull if you have an other code than this .ipynb \
You just have to extract those csv in dataframe in you own code.

In [10]:
import os
os.makedirs("datas", exist_ok=True)

for key in dataframes_dictionnary:
    dataframes_dictionnary[key].to_csv(f"datas/{key}.csv", index=False)